# Examples

This notebook gives a brief explanation and examples of how to use the `audl_advanced_stats` package.

The first time you run these functions, the data will be downloaded and stored locally, which will take a while. When you run the functions after that, the locally stored versions will be used, which is much quicker. The times in this notebook reflect the first run-through.

In [1]:
import audl_advanced_stats as audl

## Season

This section will go through the `Season` class.

In [2]:
%%time
# Initialize season object
s = audl.Season(
    year=2022,
    data_path="data",
    upload=False,
    download=False
)

# Get all players
s.get_players().head()

CPU times: user 4.93 ms, sys: 2.76 ms, total: 7.69 ms
Wall time: 7.67 ms


,player_id,player_name,team_id
0,9842,"Abrahamsson, Gavin",16
1,10473,"Akyuz, Tarik",22
2,10225,"Alongi, Robert",9
3,10058,"Anthony, Kevin",6
4,10595,"Arcata, Mike",16


In [3]:
%%time
# Get all teams
s.get_teams().head()

CPU times: user 2.35 ms, sys: 1.31 ms, total: 3.66 ms
Wall time: 2.63 ms


,team_id,team_abbrev,team_name
0,1,ATL,Atlanta Hustle
1,2,ATX,Austin Sol
2,37,BOS,Boston Glory
3,18,CAR,Carolina Flyers
4,3,CHI,Chicago Union


In [4]:
%%time
# Get all scheduled games and whether or not they've been played yet
s.get_game_info().head()

CPU times: user 2.88 ms, sys: 2.21 ms, total: 5.09 ms
Wall time: 3.76 ms


,game_date,away_team,home_team,url,events_exist,playoffs
0,2022-04-29,ATL,CAR,https://audl-stat-server.herokuapp.com/stats-p...,True,False
1,2022-04-29,SLC,SD,https://audl-stat-server.herokuapp.com/stats-p...,True,False
2,2022-04-30,ATX,DAL,https://audl-stat-server.herokuapp.com/stats-p...,True,False
3,2022-04-30,MTL,TOR,https://audl-stat-server.herokuapp.com/stats-p...,True,False
4,2022-04-30,NY,DC,https://audl-stat-server.herokuapp.com/stats-p...,True,False


In [5]:
%%time
# Compile all processed game data into a single dataframe
s.get_games().head()

CPU times: user 31 ms, sys: 116 ms, total: 147 ms
Wall time: 32.4 ms


,t,r,x,y,s,q,h,game_id,team_id,opponent_team_id,...,10466,10467,10468,10471,10472,10473,10474,10476,10478,10604
0,50,NaN,NaN,NaN,720.0,NaN,NaN,2893,18,1,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,1,NaN,NaN,NaN,NaN,NaN,NaN,2893,18,1,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,20,10159.0,13.64,19.99,NaN,NaN,NaN,2893,18,1,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,20,10141.0,-2.26,41.59,NaN,NaN,NaN,2893,18,1,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,20,10149.0,5.93,48.81,NaN,NaN,NaN,2893,18,1,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [6]:
%%time
# Get some basic QC info to determine where the raw data might be messed up
# This will typically take at least ~2-3 minutes
s.get_game_qc()

CPU times: user 3.42 s, sys: 82.2 ms, total: 3.5 s
Wall time: 3.49 s


,game_date,away_team,home_team,url,events_exist,playoffs,not_seven_players_home,not_seven_players_away,not_seven_players_home_raw,not_seven_players_away_raw,negative_elapsed_home,negative_elapsed_away,negative_time_home_raw,negative_time_away_raw,no_event_label_home,no_event_label_away,data_labels_home_raw,data_labels_away_raw
0,2022-04-29,ATL,CAR,https://audl-stat-server.herokuapp.com/stats-p...,True,False,0,0,0,0,3,0,0,0,[],[],"[h, l, ms, q, r, s, t, x, y]","[c, l, ms, q, r, s, t, x, y]"
1,2022-04-29,SLC,SD,https://audl-stat-server.herokuapp.com/stats-p...,True,False,1,0,1,0,0,0,0,0,[],[],"[c, l, ms, r, s, t, x, y]","[l, ms, q, r, s, t, x, y]"
2,2022-04-30,ATX,DAL,https://audl-stat-server.herokuapp.com/stats-p...,True,False,0,0,0,0,0,0,0,0,[],[],"[c, l, ms, q, r, s, t, x, y]","[c, l, ms, r, s, t, x, y]"
3,2022-04-30,MTL,TOR,https://audl-stat-server.herokuapp.com/stats-p...,True,False,0,0,0,0,0,0,0,0,[],[],"[c, l, ms, r, s, t, x, y]","[l, ms, r, s, t, x, y]"
4,2022-04-30,NY,DC,https://audl-stat-server.herokuapp.com/stats-p...,True,False,0,0,0,0,0,0,0,0,[],[],"[c, l, ms, r, s, t, x, y]","[c, l, ms, r, s, t, x, y]"
5,2022-04-30,PHI,BOS,https://audl-stat-server.herokuapp.com/stats-p...,True,False,0,2,0,2,0,0,0,0,[],[],"[c, h, l, ms, r, s, t, x, y]","[h, l, ms, r, s, t, x, y]"
6,2022-04-30,PIT,DET,https://audl-stat-server.herokuapp.com/stats-p...,True,False,0,0,0,0,0,0,0,0,[],[],"[l, ms, r, s, t, x, y]","[l, ms, r, s, t, x, y]"
7,2022-04-30,SLC,LA,https://audl-stat-server.herokuapp.com/stats-p...,True,False,0,0,0,0,4,0,0,0,[],[],"[h, l, ms, r, s, t, x, y]","[l, ms, r, s, t, x, y]"
8,2022-04-30,TB,ATL,https://audl-stat-server.herokuapp.com/stats-p...,True,False,0,0,0,0,0,0,0,0,[],[],"[c, h, l, ms, q, r, s, t, x, y]","[h, l, ms, q, r, s, t, x, y]"
9,2022-05-01,PIT,CHI,https://audl-stat-server.herokuapp.com/stats-p...,True,False,0,0,0,0,0,0,0,0,[],[],"[l, ms, q, r, s, t, x, y]","[l, ms, r, s, t, x, y]"


In [7]:
%%time
# Player stats for each separate game
s.get_player_stats_by_game().head()

CPU times: user 7.47 ms, sys: 6.26 ms, total: 13.7 ms
Wall time: 7.55 ms


,playerid,name,team,opponent,game_date,year,playoffs,games,total_points,o_points,...,receptions_dump_pct,completion_huck_pct,attempts_huck_pct,receptions_huck_pct,completion_swing_pct,attempts_swing_pct,receptions_swing_pct,completion_throw_pct,attempts_throw_pct,receptions_throw_pct
0,10141,Sol Yanuck,CAR,ATL,2022-04-29,2022.0,False,1.0,16.0,16.0,...,0.162791,0.5,0.081633,0.000000,1.0,0.204082,0.232558,1.0,0.408163,0.186047
1,10143,Seth Weaver,CAR,ATL,2022-04-29,2022.0,False,1.0,10.0,0.0,...,0.333333,NaN,0.000000,0.000000,1.0,0.200000,0.000000,1.0,0.400000,0.166667
2,10147,Terrence Mitchell,CAR,ATL,2022-04-29,2022.0,False,1.0,14.0,14.0,...,0.000000,NaN,0.000000,0.000000,NaN,0.000000,0.000000,1.0,0.250000,0.400000
3,10149,Henry Fisher,CAR,ATL,2022-04-29,2022.0,False,1.0,11.0,10.0,...,0.000000,1.0,0.083333,0.076923,1.0,0.083333,0.076923,1.0,0.083333,0.384615
4,10151,William Coffin,CAR,ATL,2022-04-29,2022.0,False,1.0,5.0,0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [8]:
%%time
# Aggregated player stats for the whole season
s.get_player_stats_by_season().head()

CPU times: user 7.68 ms, sys: 6.36 ms, total: 14 ms
Wall time: 8.13 ms


,playerid,name,team,year,games,total_points,o_points,o_point_scores,o_point_noturns,d_points,...,receptions_dump_pct,completion_huck_pct,attempts_huck_pct,receptions_huck_pct,completion_swing_pct,attempts_swing_pct,receptions_swing_pct,completion_throw_pct,attempts_throw_pct,receptions_throw_pct
0,10000,Noah Celuch,PIT,2022.0,2.0,11.0,2.0,1.0,0.0,9.0,...,0.000000,NaN,0.000000,0.0,1.0,0.166667,0.142857,1.00,0.166667,0.428571
1,10001,Anson Reppermund,PIT,2022.0,1.0,13.0,0.0,0.0,0.0,13.0,...,0.000000,NaN,0.000000,0.0,1.0,0.200000,0.000000,NaN,0.000000,0.500000
2,10002,Cam Bellando,PIT,2022.0,2.0,19.0,3.0,2.0,1.0,16.0,...,0.200000,0.0,0.142857,0.0,NaN,0.000000,0.100000,0.75,0.571429,0.600000
3,10003,Thomas Hansen,PIT,2022.0,2.0,11.0,2.0,1.0,0.0,9.0,...,0.000000,NaN,0.000000,0.0,NaN,0.000000,0.000000,NaN,0.000000,1.000000
4,10004,Andrew Thompson,PIT,2022.0,1.0,6.0,1.0,1.0,1.0,5.0,...,0.333333,NaN,0.000000,0.0,NaN,0.000000,0.333333,0.50,0.666667,0.000000


In [9]:
%%time
# Heatmap and histograms
fighm, fighy, fighx = s.visual_field_heatmap_vertical(
    outcome_measure="throw_outcome",
    outcome="Completion",
    metric="count"
)

# Combine heatmap and histograms into single plot
# The aspect ratio is weird here because it's handled in the app using html/css
s.visual_field_heatmap_subplots_vertical(fighm, fighx, fighy)

CPU times: user 275 ms, sys: 60.1 ms, total: 335 ms
Wall time: 477 ms


## Game

This section will go through the `Game` class using the first game of the season (DC at NY) as an example.

In [10]:
g = audl.Game("https://audl-stat-server.herokuapp.com/stats-pages//game/2022-04-30-NY-DC")
g.get_player_stats_by_game(home=False).head().iloc[:, 20:40]

/Users/johnlithio/Documents/AUDL-Advanced-Stats/audl_advanced_stats/game.py:2380: FutureWarning:

Dropping of nuisance columns in DataFrame reductions (with 'numeric_only=None') is deprecated; in a future version this will raise TypeError.  Select only valid columns before calling the reduction.

/Users/johnlithio/Documents/AUDL-Advanced-Stats/audl_advanced_stats/game.py:2781: FutureWarning:

Dropping of nuisance columns in DataFrame reductions (with 'numeric_only=None') is deprecated; in a future version this will raise TypeError.  Select only valid columns before calling the reduction.



,o_possession_scores,d_possessions,d_possession_scores_allowed,possessions_team,o_possessions_team,d_possessions_team,minutes_played,throwaways,completions,receptions,drops,stalls,assists,goals,blocks,callahans,throwaways_team,drops_team,completions_team,stalls_team
0,18,2,1,59,31,29,22.833333,2.0,36.0,40.0,1.0,0.0,7.0,2.0,0.0,0.0,4,1,225,0
1,18,5,3,59,31,29,24.333333,1.0,20.0,25.0,0.0,0.0,4.0,4.0,0.0,0.0,5,1,226,0
2,0,9,5,59,31,29,8.166667,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1,0,6,0
3,16,2,1,59,31,29,20.566667,0.0,22.0,24.0,0.0,0.0,2.0,1.0,0.0,0.0,4,1,214,0
4,3,16,10,59,31,29,16.450000,0.0,10.0,8.0,0.0,0.0,0.0,0.0,0.0,0.0,1,0,36,0


In [11]:
g.get_response()

{'game': {'id': 2809,
  'team_season_id_home': 240,
  'team_season_id_away': 239,
  'status_id': 100,
  'score_home': 19,
  'score_away': 22,
  'live': None,
  'reg_season': True,
  'ignore_game': False,
  'lock': True,
  'start_timestamp': '2022-04-30T19:00:00.000Z',
  'start_timezone': 'EDT',
  'start_time_tbd': None,
  'aw_section': 'week-1',
  'score_times_home': [0,
   52,
   122,
   294,
   476,
   618,
   881,
   1051,
   1271,
   1360,
   1540,
   1613,
   1888,
   2160,
   2275,
   2475,
   2509,
   2632,
   2774,
   2852],
  'score_times_away': [0,
   79,
   197,
   397,
   577,
   657,
   769,
   857,
   957,
   1142,
   1404,
   1578,
   1662,
   1774,
   1998,
   2041,
   2117,
   2233,
   2545,
   2676,
   2714,
   2823,
   2880],
  'ext_game_id': '2022-04-30-NY-DC',
  'update_timestamp': '2022-05-01T01:06:16.829Z',
  'location_id': 13,
  'ls_game_id': '2dc34d97-a4d5-4175-8421-526e472e2642',
  'ticket_url': 'https://shop.thedcbreeze.com/collections/tickets',
  'streaming_

In [12]:
%%time
# Initialize game object using URL from season object
g = audl.Game(s.get_game_info()["url"].iloc[0])

CPU times: user 238 µs, sys: 40 µs, total: 278 µs
Wall time: 262 µs


In [13]:
%%time
# Here's what the raw events look like
# Each dict in the list represents a different event. For example, 20 is a completion, 21 is a score, etc.
# See constants.py for a list of the conversions.
g.get_home_events_raw()

CPU times: user 83.1 ms, sys: 1.87 ms, total: 85 ms
Wall time: 84.5 ms


[{'t': 50},
 {'t': 1, 'l': [10141, 10147, 10149, 10154, 10158, 10159, 10174]},
 {'t': 20, 'r': 10159, 'x': 13.64, 'y': 19.99},
 {'t': 20, 'r': 10141, 'x': -2.26, 'y': 41.59},
 {'t': 20, 'r': 10149, 'x': 5.93, 'y': 48.81},
 {'t': 20, 'r': 10141, 'x': 8.52, 'y': 38.52},
 {'t': 20, 'r': 10154, 'x': 23.25, 'y': 44.11},
 {'t': 20, 'r': 10141, 'x': 0.13, 'y': 39.61},
 {'t': 20, 'r': 10174, 'x': -16.03, 'y': 45.88},
 {'t': 20, 'r': 10149, 'x': -19.92, 'y': 61.15},
 {'t': 20, 'r': 10141, 'x': -2.05, 'y': 51.2},
 {'t': 22, 'r': 10158, 'x': 24.75, 'y': 108.61, 's': 678},
 {'t': 2, 'l': [10153, 10160, 10162, 10171, 10167, 10176, 10157]},
 {'t': 3, 'x': 0, 'y': 80, 'ms': 0},
 {'t': 5, 'r': 10160, 'q': 1},
 {'t': 20, 'r': 10160, 'x': -9.96, 'y': 65.52},
 {'t': 20, 'r': 10162, 'x': -7.37, 'y': 76.09},
 {'t': 20, 'r': 10171, 'x': 21.68, 'y': 82.56},
 {'t': 20, 'r': 10162, 'x': 22.64, 'y': 87.06},
 {'t': 20, 'r': 10171, 'x': 6.61, 'y': 90},
 {'t': 8, 'x': -23.81, 'y': 98.31},
 {'t': 11},
 {'t': 21, 's

In [14]:
%%time
# Here's what the processed events look like
g.get_home_events().head()

CPU times: user 5.65 ms, sys: 2.65 ms, total: 8.3 ms
Wall time: 5.28 ms


,t,r,x,y,s,q,h,game_id,team_id,opponent_team_id,...,10165,10166,10167,10171,10173,10174,10176,centering_pass,throw_type,play_description
0,50,NaN,NaN,NaN,720.0,NaN,NaN,2893,18,1,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,False,None,
1,1,NaN,NaN,NaN,NaN,NaN,NaN,2893,18,1,...,0.0,0.0,0.0,0.0,0.0,1.0,0.0,False,None,
2,20,10159.0,13.64,19.99,NaN,NaN,NaN,2893,18,1,...,0.0,0.0,0.0,0.0,0.0,1.0,0.0,True,Throw,Completion: Matt Gouchoe-Hanas Throw to<br>Sol...
3,20,10141.0,-2.26,41.59,NaN,NaN,NaN,2893,18,1,...,0.0,0.0,0.0,0.0,0.0,1.0,0.0,False,Dish,Completion: Sol Yanuck Dish to<br>Henry Fisher...
4,20,10149.0,5.93,48.81,NaN,NaN,NaN,2893,18,1,...,0.0,0.0,0.0,0.0,0.0,1.0,0.0,False,Dump,Completion: Henry Fisher Dump to<br>Sol Yanuck...


In [15]:
%%time
# We can also get some basic info about the game
g.get_game_info()

CPU times: user 4 µs, sys: 1 µs, total: 5 µs
Wall time: 8.11 µs


,id,team_season_id_home,team_season_id_away,status_id,score_home,score_away,live,reg_season,ignore_game,lock,...,aw_section,ext_game_id,update_timestamp,location_id,ls_game_id,ticket_url,streaming_url,roster_set,roster_set_1,roster_set_2
0,2893,242,241,100,16,14,None,True,False,True,...,week-1,2022-04-29-ATL-CAR,2022-04-30T00:55:50.838Z,53,55790353-0270-4b7c-b916-948dd69b9abb,https://theaudl.com/flyers/tickets,https://audltv.vhx.tv/videos/atlanta-at-caroli...,True,True,True


In [16]:
%%time
# We can also get some basic info about each team
g.get_home_team()

CPU times: user 3.72 ms, sys: 551 µs, total: 4.27 ms
Wall time: 3.83 ms


,abbrev,city,division_id,final_standing,id,season_id,team_id,ulti_analytics_ext_id,name,ext_team_id,ls_team_id
0,CAR,Carolina,4,None,242,13,18,None,Flyers,flyers,f364b191-1101-4c86-959e-7ac9b2a0909b


In [17]:
%%time
# We can also get the rosters and which players were active
g.get_home_roster().head()

CPU times: user 6.97 ms, sys: 1.28 ms, total: 8.24 ms
Wall time: 7.33 ms


,id,team_season_id,player_id,jersey_number,first_name,last_name,ext_player_id,ls_player_id,active
0,10139,242,2889,0,Wilson,Matthews,wmatthews1,8bdcaa7f-035c-4f60-a1b3-2b5352519ada,False
1,10140,242,994,1,Justin,Allen,jallen,62b8c47d-76a7-4eae-a397-a313327407be,False
2,10141,242,1643,2,Sol,Yanuck,syanuck,4338c80e-2abd-40e8-a32a-0b95e347aba3,True
3,10142,242,2890,3,Michael,Lee,mlee1,d56afe48-2b65-4de1-8594-5650ea403a92,False
4,10143,242,2222,4,Seth,Weaver,sweaver,3264b70f-0b34-4733-a3e7-56173decc733,True


In [18]:
%%time
# Get player stats for this game
g.get_player_stats_by_game(home=True).head()

/Users/johnlithio/Documents/AUDL-Advanced-Stats/audl_advanced_stats/game.py:2380: FutureWarning:

Dropping of nuisance columns in DataFrame reductions (with 'numeric_only=None') is deprecated; in a future version this will raise TypeError.  Select only valid columns before calling the reduction.

/Users/johnlithio/Documents/AUDL-Advanced-Stats/audl_advanced_stats/game.py:2781: FutureWarning:

Dropping of nuisance columns in DataFrame reductions (with 'numeric_only=None') is deprecated; in a future version this will raise TypeError.  Select only valid columns before calling the reduction.



CPU times: user 1.14 s, sys: 11.9 ms, total: 1.15 s
Wall time: 1.14 s


,playerid,name,team,opponent,game_date,year,playoffs,games,total_points,o_points,...,receptions_dump_pct,completion_huck_pct,attempts_huck_pct,receptions_huck_pct,completion_swing_pct,attempts_swing_pct,receptions_swing_pct,completion_throw_pct,attempts_throw_pct,receptions_throw_pct
0,10141,Sol Yanuck,CAR,ATL,2022-04-29,2022,False,1,16,16,...,0.162791,0.5,0.081633,0.000000,1.0,0.204082,0.232558,1.0,0.408163,0.186047
1,10143,Seth Weaver,CAR,ATL,2022-04-29,2022,False,1,10,0,...,0.333333,NaN,0.000000,0.000000,1.0,0.200000,0.000000,1.0,0.400000,0.166667
2,10147,Terrence Mitchell,CAR,ATL,2022-04-29,2022,False,1,14,14,...,0.000000,NaN,0.000000,0.000000,NaN,0.000000,0.000000,1.0,0.250000,0.400000
3,10149,Henry Fisher,CAR,ATL,2022-04-29,2022,False,1,11,10,...,0.000000,1.0,0.083333,0.076923,1.0,0.083333,0.076923,1.0,0.083333,0.384615
4,10151,William Coffin,CAR,ATL,2022-04-29,2022,False,1,5,0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [19]:
%%time
# Visualize the progression of scoring
g.visual_game_score()

CPU times: user 86.1 ms, sys: 8.23 ms, total: 94.3 ms
Wall time: 97.9 ms


In [20]:
%%time
# Visualize the substitution patterns
g.visual_game_flow()

CPU times: user 309 ms, sys: 7.29 ms, total: 316 ms
Wall time: 329 ms


In [22]:
%%time
# Visualize a single possession
g.visual_possession_map_vertical(possession_number=1)

CPU times: user 130 ms, sys: 24.5 ms, total: 154 ms
Wall time: 174 ms
